# IMPORTS AND INSTALLS

In [1]:
!pip3 install smogn

  Using cached smogn-0.1.2-py3-none-any.whl (30 kB)


In [3]:
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler as MMS
from sklearn.preprocessing import StandardScaler as SS
import re
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn import metrics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.decomposition import PCA
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
import smogn

In [4]:
ss=SS()

# DATA CLEANING AND SYNTHETIC DATA PREP
**df -> synthetic dataframe**<br>
**df_org-> orginal dataframe**

In [6]:
df_org=pd.read_csv('C:\\Users\\KARAN\\Desktop\\diet project\\train_data_329x19.csv')
df_org.drop(axis=1,labels='Unnamed: 0',inplace=True)
df_org.dropna(axis=0,inplace=True)
df_org.reset_index(drop=True,inplace=True)
df_org['patient_age']=df_org['patient_age'].astype(int)
df = smogn.smoter(
    
    ## main arguments
    data = df_org,           ## pandas dataframe
    y = 'Fat',          ## string ('header name')
    k =10,                    ## positive integer (k < n)
    samp_method = 'extreme',  ## string ('balance' or 'extreme')

    ## phi relevance arguments
    rel_thres = 0.8,         ## positive real number (0 < R < 1)
    rel_method = 'auto',      ## string ('auto' or 'manual')
    rel_xtrm_type = 'high',   ## string ('low' or 'both' or 'high')
    rel_coef = 2.25          ## positive real number (0 < R)
)
df= df.round(2)
x=df.iloc[:,:13]

y=df.iloc[:,15:16]
y
x_val=df_org.iloc[:,:13]

y_val=df_org.iloc[:,15:16]
df.shape


synth_matrix: 100%|##########| 3/3 [00:00<00:00,  5.04it/s]


(643, 19)

In [7]:
y.describe()

,Fat
count,643.000000
mean,34.202644
std,7.591895
min,2.500000
25%,26.550000
50%,36.000000
75%,41.990000
max,42.050000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 0 to 324
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   patient_age     643 non-null    int32  
 1   patient_gender  643 non-null    int64  
 2   Ht              643 non-null    float64
 3   Wt              643 non-null    float64
 4   HB              643 non-null    float64
 5   UREA            643 non-null    float64
 6   CR              643 non-null    float64
 7   Na              643 non-null    float64
 8   K               643 non-null    float64
 9   FBS             643 non-null    float64
 10  HBA1C           643 non-null    float64
 11  SGOT            643 non-null    float64
 12  SGPT            643 non-null    float64
 13  calories        643 non-null    float64
 14  cho             643 non-null    float64
 15  Fat             643 non-null    float64
 16  pro             643 non-null    float64
 17  fluids          643 non-null    flo

In [9]:
df_org.describe()

,patient_age,patient_gender,Ht,Wt,HB,UREA,CR,Na,K,FBS,HBA1C,SGOT,SGPT,calories,cho,Fat,pro,fluids,NA
count,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.00000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000
mean,55.652308,0.606154,159.876615,66.876862,13.294492,84.096277,1.038708,134.978462,4.088862,83.846154,6.49280,24.186154,26.158954,1659.036923,235.395385,27.226154,66.273846,29.055692,5.673846
std,18.460478,0.489355,6.428141,9.200941,2.177099,36.993946,0.794877,2.754821,0.350768,10.946321,1.91834,10.665496,12.622462,247.451152,112.675699,3.823283,7.452448,136.173058,1.828637
min,10.000000,0.000000,135.000000,25.900000,6.580000,10.000000,0.530000,124.000000,2.400000,83.000000,4.60000,7.500000,6.700000,150.000000,164.000000,2.500000,43.000000,1.000000,0.000000
25%,43.000000,0.000000,156.000000,63.000000,12.300000,41.400000,0.860000,135.000000,4.100000,83.000000,5.29000,22.800000,24.800000,1520.000000,209.000000,25.000000,62.000000,2.500000,5.000000
50%,58.000000,1.000000,160.000000,68.000000,13.700000,106.040000,0.890000,135.000000,4.100000,83.000000,5.80000,22.800000,24.800000,1610.000000,221.000000,26.800000,67.000000,2.900000,6.000000
75%,70.000000,1.000000,163.500000,71.700000,14.800000,106.970000,0.960000,135.000000,4.100000,83.000000,6.97000,22.800000,24.800000,1820.000000,250.000000,30.000000,70.000000,3.000000,7.000000
max,94.000000,1.000000,180.000000,98.500000,17.300000,200.700000,9.780000,157.000000,6.700000,276.000000,14.60000,133.000000,170.000000,2500.000000,2180.000000,42.000000,90.000000,800.000000,8.000000


In [10]:
df.describe()

,patient_age,patient_gender,Ht,Wt,HB,UREA,CR,Na,K,FBS,HBA1C,SGOT,SGPT,calories,cho,Fat,pro,fluids,NA
count,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000
mean,33.472784,0.800933,150.625210,52.119005,12.126392,69.235179,0.925661,135.152939,4.176579,83.427683,5.962333,24.241400,24.148974,2058.471120,286.801166,34.202644,69.435692,16.252208,6.824261
std,26.001326,0.399609,10.880601,16.337632,2.129616,40.303265,0.581635,2.440627,0.312271,7.787820,1.492742,7.722819,9.580251,441.942267,95.572019,7.591895,6.430097,97.601789,1.744212
min,9.000000,0.000000,135.000000,25.900000,6.580000,10.000000,0.530000,124.000000,2.400000,83.000000,4.600000,7.500000,6.700000,150.000000,164.000000,2.500000,43.000000,1.000000,0.000000
25%,11.000000,1.000000,138.065000,37.200000,11.250000,28.000000,0.850000,134.930000,4.100000,83.000000,5.080000,22.800000,24.700000,1610.000000,221.000000,26.550000,67.000000,2.510000,6.000000
50%,12.000000,1.000000,147.630000,38.980000,11.890000,105.050000,0.890000,135.000000,4.100000,83.000000,5.660000,22.800000,24.800000,2230.000000,329.760000,36.000000,70.030000,3.000000,8.000000
75%,59.000000,1.000000,160.000000,68.000000,13.715000,106.525000,0.900000,135.060000,4.110000,83.000000,5.970000,22.850000,24.800000,2499.365000,342.995000,41.990000,73.000000,3.500000,8.000000
max,94.000000,1.000000,180.000000,98.500000,17.300000,200.700000,9.780000,157.000000,6.700000,276.000000,14.600000,133.000000,170.000000,2502.870000,2180.000000,42.050000,90.000000,800.000000,8.000000


In [11]:
df.to_csv('oversampled_data.csv')

# TEST TRAIN SPLIT

In [12]:
x_train,x_test,y_train,y_test=train_test_split(x_val,y_val,test_size=0.3)

In [13]:
lr=LinearRegression(fit_intercept=True)
en=ElasticNet()
rf=RandomForestRegressor(n_estimators=500,max_depth=3,criterion='absolute_error')
ab=AdaBoostRegressor(n_estimators=100)
gb=GradientBoostingRegressor(n_estimators=100,max_depth=1)
pca=PCA(whiten=True)
lda=LDA()
gpr=GaussianProcessRegressor(kernel=WhiteKernel()+DotProduct())
lgr=LogisticRegression()
svr=SVR(gamma='auto',kernel='rbf')

In [14]:
lr.fit(x_train,y_train.values.ravel())
en.fit(x_train,y_train.values.ravel())
rf.fit(x_train,y_train.values.ravel())
ab.fit(x_train,y_train.values.ravel())
gb.fit(x_train,y_train.values.ravel())
pca.fit(x_train,y_train.values.ravel())
#lda.fit(x_tr,y_tr.values.reshape(-1,1))
gpr.fit(x_train,y_train.values.ravel())
svr.fit(x_train,y_train.values.ravel())

SVR(gamma='auto')

# TRAIN ACC WITH ORGINAL DATASET

In [15]:
print("TRAIN ACC:")
#fh.write("TRAIN ACC - BEFORE FEATURE SELECTION:\n")
print('LR acc: ',lr.score(x_train,y_train.values.ravel()))
print('ElasticNet acc: ',en.score(x_train,y_train.values.ravel()))
print('RandomForest acc: ',rf.score(x_train,y_train.values.ravel()))
print('AdaBoost acc: ',ab.score(x_train,y_train.values.ravel()))
print('GradientBoost acc: ',gb.score(x_train,y_train.values.ravel()))
#fh.write("Gradient Boosting acc: "+str(gb.score(x_train,y_train.values.ravel()))+"\n")

print("PCA acc: ",pca.score(x_train,y_train.values.ravel()))
print("GPR acc: ",gpr.score(x_train,y_train.values.ravel()))
print("SVR acc: ",svr.score(x_train,y_train.values.ravel()))


TRAIN ACC:
LR acc:  0.5214983224912663
ElasticNet acc:  0.38715630084171404
RandomForest acc:  0.6263113928715607
AdaBoost acc:  0.8306975001773171
GradientBoost acc:  0.6886249365489496
PCA acc:  -54.97599516454304
GPR acc:  0.5156296148351678
SVR acc:  0.2989681843148835


# TEST ACC AFTER TRAIN WITH ORGINAL DATASET

In [16]:
print("TEST ACC:")
#fh.write("TEST ACC - BEFORE FEATURE SELECTION:\n")

print('LR acc: ',lr.score(x_test,y_test.values.ravel()))
print('ElasticNet acc: ',en.score(x_test,y_test.values.ravel()))
print('RandomForest acc: ',rf.score(x_test,y_test.values.ravel()))
print('AdaBoost acc: ',ab.score(x_test,y_test.values.ravel()))
print('GradientBoost acc: ',gb.score(x_test,y_test.values.ravel()))
#fh.write("Gradient Boosting acc: "+str(gb.score(x_test,y_test.values.ravel()))+"\n")
print("PCA acc: ",pca.score(x_test,y_test.values.ravel()))
print("GPR acc: ",gpr.score(x_test,y_test.values.ravel()))
print("SVR acc: ",svr.score(x_test,y_test.values.ravel()))


TEST ACC:
LR acc:  0.27784838183150495
ElasticNet acc:  0.24493112804992556
RandomForest acc:  0.5061527533727905
AdaBoost acc:  0.36851161611240546
GradientBoost acc:  0.49896398792996066
PCA acc:  -59.16789004012572
GPR acc:  0.294864398307769
SVR acc:  0.06165435757339066


# TRAIN SYNTHETIC TEST ORGINAL

In [17]:
lr=LinearRegression(fit_intercept=True)
en=ElasticNet()
rf=RandomForestRegressor(n_estimators=500,max_depth=3,criterion='absolute_error')
ab=AdaBoostRegressor(n_estimators=100)
gb=GradientBoostingRegressor(n_estimators=100,max_depth=1)
pca=PCA(whiten=True)
lda=LDA()
gpr=GaussianProcessRegressor(kernel=WhiteKernel()+DotProduct())
lgr=LogisticRegression()
svr=SVR(gamma='auto',kernel='rbf')

In [18]:
#x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0);

In [19]:
x

,patient_age,patient_gender,Ht,Wt,HB,UREA,CR,Na,K,FBS,HBA1C,SGOT,SGPT
0,11,1,138.07,37.20,9.21,39.07,0.89,132.96,4.60,83.0,5.30,22.83,24.82
1,11,1,137.99,37.17,9.19,37.58,0.89,132.99,4.59,83.0,5.31,22.79,24.75
2,12,1,137.95,37.18,9.19,36.85,0.89,133.03,4.61,83.0,5.30,22.73,24.81
3,12,1,137.94,37.19,9.23,38.04,0.89,132.98,4.61,83.0,5.30,22.79,24.64
4,12,1,138.06,37.24,9.20,38.81,0.89,132.96,4.60,83.0,5.31,22.79,24.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,57,1,160.00,51.00,13.91,105.15,0.89,135.00,4.10,83.0,5.80,22.80,24.80
321,27,1,164.00,74.00,16.11,105.92,0.86,135.00,4.10,83.0,5.18,22.80,24.80
322,48,1,159.00,68.50,16.57,105.60,0.87,135.00,4.10,83.0,7.31,22.80,24.80
323,85,1,163.00,78.00,16.43,107.76,0.85,135.00,4.10,83.0,5.80,22.80,24.80


In [20]:
y

,Fat
0,40.00
1,40.01
2,40.00
3,40.01
4,40.01
...,...
320,27.00
321,30.00
322,31.00
323,28.00


In [21]:
lr.fit(x,y.values.ravel())
en.fit(x,y.values.ravel())
rf.fit(x,y.values.ravel())
ab.fit(x,y.values.ravel())
gb.fit(x,y.values.ravel())
pca.fit(x,y.values.ravel())
#lda.fit(x_tr,y_tr.values.reshape(-1,1))
gpr.fit(x,y.values.ravel())
svr.fit(x,y.values.ravel())

SVR(gamma='auto')

# ACC WITH SYNTHETIC DATA

In [22]:
print('LR acc: ',lr.score(x,y.values.ravel()))
print('ElasticNet acc: ',en.score(x,y.values.ravel()))
print('RandomForest acc: ',rf.score(x,y.values.ravel()))
print('AdaBoost acc: ',ab.score(x,y.values.ravel()))
print('GradientBoost acc: ',gb.score(x,y.values.ravel()))

#fh.write("Gradient Boosting acc: "+str(gb.score(x,y.values.ravel()))+"\n")
print("PCA acc: ",pca.score(x,y.values.ravel()))
print("GPR acc: ",gpr.score(x,y.values.ravel()))
print("SVR acc: ",svr.score(x,y.values.ravel()))

LR acc:  0.8965187964182599
ElasticNet acc:  0.8481633472316117
RandomForest acc:  0.9473045450253531
AdaBoost acc:  0.9633691942681729
GradientBoost acc:  0.9477261907218301
PCA acc:  -55.61090540761919
GPR acc:  0.8962193235690666
SVR acc:  0.9314703230589608


# ACC WITH ACTUAL DATA

In [23]:
print('LR acc: ',lr.score(x_val,y_val.values.ravel()))
print('ElasticNet acc: ',en.score(x_val,y_val.values.ravel()))
print('RandomForest acc: ',rf.score(x_val,y_val.values.ravel()))
print('AdaBoost acc: ',ab.score(x_val,y_val.values.ravel()))
print('GradientBoost acc: ',gb.score(x_val,y_val.values.ravel()))

#fh.write("Gradient Boosting acc: "+str(gb.score(x_val,y_val.values.ravel()))+"\n")
print("PCA acc: ",pca.score(x_val,y_val.values.ravel()))
print("GPR acc: ",gpr.score(x_val,y_val.values.ravel()))
print("SVR acc: ",svr.score(x_val,y_val.values.ravel()))

LR acc:  0.24276105686059868
ElasticNet acc:  -0.10222185333017308
RandomForest acc:  0.5883121825637057
AdaBoost acc:  0.7174423911670722
GradientBoost acc:  0.6071413506601988
PCA acc:  -66.15937446558756
GPR acc:  0.24083190864410287
SVR acc:  0.4793433719580561


# RMSE WHEN PREDICTING WITH ACTUAL DATA

In [24]:

print('LR acc: ',metrics.mean_squared_error(y_val,lr.predict(x_val))**0.5)
print('ElasticNet acc: ',metrics.mean_squared_error(y_val,en.predict(x_val))**0.5)
print('RandomForest acc: ',metrics.mean_squared_error(y_val,rf.predict(x_val))**0.5)
print('AdaBoost acc: ',metrics.mean_squared_error(y_val,ab.predict(x_val))**0.5)
print('GradientBoost acc: ',metrics.mean_squared_error(y_val,gb.predict(x_val))**0.5)

#fh.write("Gradient Boosting acc: "+str(metrics.mean_squared_error(y_val,lr.predict(x_val))**0.5)+"\n")
#print("PCA acc: ",metrics.mean_squared_error(y_val,pca.predict(x_val)**0.5))
print("GPR acc: ",metrics.mean_squared_error(y_val,gpr.predict(x_val)**0.5))
print("SVR acc: ",metrics.mean_squared_error(y_val,svr.predict(x_val)**0.5))

LR acc:  3.321878474116595
ElasticNet acc:  4.0077606673946375
RandomForest acc:  2.449352423978915
AdaBoost acc:  2.0291809354889545
GradientBoost acc:  2.3926844639325537
GPR acc:  496.3625602829155
SVR acc:  497.8532893949407


# CONCLUSION:
- Boosting models perform better
- training with synthetic data and predicting with the actual data provides better results than just training and predicting with the original dataset 